In [1]:
import psycopg2
import pandas as pd
import datetime

from login import LOGIN_KWARGS

In [2]:
conn = psycopg2.connect(**LOGIN_KWARGS)

In [3]:
cur = conn.cursor()
cur.execute("begin transaction")

In [5]:
cur.execute("select distinct username from votes")
cur.fetchall()

[('48297413',),
 ('27927048',),
 ('99497524',),
 ('38249914',),
 ('4321',),
 ('19038446',),
 ('19103958',)]

In [9]:
cur.execute("select COUNT(*) from votes where length(username) = 8")
cur.fetchall()

[(1390,)]

In [4]:
cur.execute("select column_name, ordinal_position from information_schema.columns where table_name='users' order by ordinal_position")
cur.fetchall()

[('username', 1),
 ('age', 2),
 ('gender', 4),
 ('country', 5),
 ('region', 6),
 ('ethnicity', 7)]

In [5]:
cur.execute("select column_name, ordinal_position from information_schema.columns where table_name='votes' order by ordinal_position")
cur.fetchall()

[('username', 1),
 ('chosen_image_filename', 2),
 ('other_image_filename', 3),
 ('timestamp', 4)]

In [ ]:
cur.execute("select * from users")
cur.fetchall()

In [63]:
cur.execute("select table_name, ordinal_position, data_type, column_name from information_schema.columns where table_name='votes' or table_name='users' order by table_name, ordinal_position")
res = cur.fetchall()
pd.DataFrame(res)

,0,1,2,3
0,users,1,character varying,username
1,users,2,integer,age
2,users,4,USER-DEFINED,gender
3,votes,1,character varying,username
4,votes,2,character varying,chosen_image_filename
5,votes,3,character varying,other_image_filename
6,votes,4,timestamp without time zone,time


In [8]:
cur.execute("insert into users (username, age, gender) values (%s, %s, %s)", ("bob", 9, "Male"))

UniqueViolation: duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (username)=(bob) already exists.


In [9]:
cur.execute("insert into votes (username, chosen_image_filename, other_image_filename, time) values (%s, %s, %s, %s)", ("bob", "a.png", "b.png", datetime.datetime.now()))

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [96]:
conn.commit()

In [6]:
cur.execute("select * from users")
cur.fetchall()

[('bob', 9, 'Male', None, None, None),
 ('132435', None, None, None, None, None),
 ('test123', 48, 'Male', None, None, None),
 ('test12345', 59, 'Male', None, None, None),
 ('test1', 22, 'Male', None, None, None),
 ('4321', 21, 'Male', 'United Kingdom', 'Cambridgeshire', 'English'),
 ('123456', 21, 'Male', 'Uzbekistan', 'Namangan', 'English')]

In [101]:
cur.execute("abort transaction")

In [49]:
# Tables: users, votes
cur.execute("select column_name, ordinal_position from information_schema.columns where table_name='columns' order by ordinal_position")
cur.fetchall()

[('table_catalog', 1),
 ('table_schema', 2),
 ('table_name', 3),
 ('column_name', 4),
 ('ordinal_position', 5),
 ('column_default', 6),
 ('is_nullable', 7),
 ('data_type', 8),
 ('character_maximum_length', 9),
 ('character_octet_length', 10),
 ('numeric_precision', 11),
 ('numeric_precision_radix', 12),
 ('numeric_scale', 13),
 ('datetime_precision', 14),
 ('interval_type', 15),
 ('interval_precision', 16),
 ('character_set_catalog', 17),
 ('character_set_schema', 18),
 ('character_set_name', 19),
 ('collation_catalog', 20),
 ('collation_schema', 21),
 ('collation_name', 22),
 ('domain_catalog', 23),
 ('domain_schema', 24),
 ('domain_name', 25),
 ('udt_catalog', 26),
 ('udt_schema', 27),
 ('udt_name', 28),
 ('scope_catalog', 29),
 ('scope_schema', 30),
 ('scope_name', 31),
 ('maximum_cardinality', 32),
 ('dtd_identifier', 33),
 ('is_self_referencing', 34),
 ('is_identity', 35),
 ('identity_generation', 36),
 ('identity_start', 37),
 ('identity_increment', 38),
 ('identity_maximum', 39)

In [73]:

cur.execute("select column_name, ordinal_position from information_schema.columns where table_name='tables' order by ordinal_position")
cur.fetchall()

[('table_catalog', 1),
 ('table_schema', 2),
 ('table_name', 3),
 ('table_type', 4),
 ('self_referencing_column_name', 5),
 ('reference_generation', 6),
 ('user_defined_type_catalog', 7),
 ('user_defined_type_schema', 8),
 ('user_defined_type_name', 9),
 ('is_insertable_into', 10),
 ('is_typed', 11),
 ('commit_action', 12)]

In [81]:

cur.execute("select column_name, ordinal_position from information_schema.columns where table_name='user_defined_types' order by ordinal_position")
cur.fetchall()

[('user_defined_type_catalog', 1),
 ('user_defined_type_schema', 2),
 ('user_defined_type_name', 3),
 ('user_defined_type_category', 4),
 ('is_instantiable', 5),
 ('is_final', 6),
 ('ordering_form', 7),
 ('ordering_category', 8),
 ('ordering_routine_catalog', 9),
 ('ordering_routine_schema', 10),
 ('ordering_routine_name', 11),
 ('reference_type', 12),
 ('data_type', 13),
 ('character_maximum_length', 14),
 ('character_octet_length', 15),
 ('character_set_catalog', 16),
 ('character_set_schema', 17),
 ('character_set_name', 18),
 ('collation_catalog', 19),
 ('collation_schema', 20),
 ('collation_name', 21),
 ('numeric_precision', 22),
 ('numeric_precision_radix', 23),
 ('numeric_scale', 24),
 ('datetime_precision', 25),
 ('interval_type', 26),
 ('interval_precision', 27),
 ('source_dtd_identifier', 28),
 ('ref_dtd_identifier', 29)]

# Final code

In [23]:
def get_cols(cur):
    cur.execute("select column_name from information_schema.columns where table_name='votes' order by table_name, ordinal_position")
    return [x for (x,) in cur.fetchall()]

def get_vote_df(cur):
    cur.execute("select * from votes")
    votes = cur.fetchall()
    cols = get_cols(cur)
    data = []
    for vote in votes:
        assert len(vote) == len(cols)
        res = {}
        for name, val in zip(cols, vote):
            res[name] = val
        data.append(res)
    return pd.DataFrame(data)

In [24]:
with psycopg2.connect(amogus) as conn2:
    with conn2.cursor() as cur2:
        votes = get_vote_df(cur2)

votes

,username,chosen_image_filename,other_image_filename,timestamp
0,bob,a.png,b.png,2023-02-02 12:59:58.047640
1,123456,100_1_1.png,100_1_3.png,2023-02-05 21:55:33.564580
2,123456,100_1_4.png,100_1_1.png,2023-02-05 21:55:33.564580
3,123456,100_1_1.png,100_1_5.png,2023-02-05 21:55:33.564580
4,123456,100_1_6.png,100_1_1.png,2023-02-05 21:55:33.564580
...,...,...,...,...
89,test1,100_1_1.png,100_1_3.png,2023-02-07 13:49:13.468991
90,test1,100_1_4.png,100_1_1.png,2023-02-07 13:49:13.468991
91,test1,100_1_1.png,100_1_5.png,2023-02-07 13:49:13.468991
92,test1,100_1_6.png,100_1_1.png,2023-02-07 13:49:13.468991


In [25]:
def process_votes(df):
    unique_images = pd.concat([df["chosen_image_filename"], df["other_image_filename"]]).unique()
    unique_images.sort()
    return unique_images

process_votes(votes)

array(['100_1_1.png', '100_1_10.png', '100_1_2.png', '100_1_3.png',
       '100_1_4.png', '100_1_5.png', '100_1_6.png', '100_1_7.png',
       '100_1_8.png', '100_1_9.png', '106_1_1.png', '106_1_10.png',
       '106_1_2.png', '106_1_3.png', '106_1_4.png', '106_1_5.png',
       '106_1_6.png', '106_1_7.png', '106_1_8.png', '106_1_9.png',
       'a.png', 'b.png'], dtype=object)

# Meeting notes

- If we remove features for each drawing, we suddenly end up with a lot of pairwise votes again. What can we do to fix this?
A: Nothing. We have enough vots just within our group anyway :P